 # 1. Generate MESH and INCON files
 In this notebook, we assume that the mesh has already been generated by a third-party software (here Gmsh via `pygmsh`, see notebook "0. Generate mesh in Python with Gmsh").

 First, we import `numpy` and `toughio`.

In [1]:
import numpy
import toughio


 A supported mesh can be read using the function `toughio.read_mesh` that returns a `toughio.Mesh` object that follows the same convention as `meshio >= 4.0, < 5.0`. More specifically, all the cell properties of the mesh (e.g. `cells`, values of `cell_data`...) return a list of `array_like`. This design choice has been kept in `toughio` for the sake of consistency with `meshio` since it mostly relies on it for mesh reading/writing. Fully ordered cell data arrays can be reconstructed using the function ``numpy.concatenate``.

In [2]:
mesh = toughio.read_mesh("mesh.msh")


 The mesh used in this sample problem is 2D and has been defined in the XY plane, but the points have 3D coordinates (with zeros as 3rd dimension for every cells). To make it 3D in the XZ plane, we swap the 2nd and 3rd dimensions, and then extrude the mesh by 1 meter along the Y axis (2nd dimension).

In [3]:
mesh.points[:, [1, 2]] = mesh.points[:, [2, 1]]
mesh.extrude_to_3d(height=1.0, axis=1)


 We start by defining the boundary conditions. `toughio` recognizes the cell data key "boundary_condition" and automatically imposes Dirichlet boundary conditions to cells that have any value other than 0 in this cell data array.
 In this example, we simply set 1 to cells that belong to the group "BOUND" and 0 to others.

In [4]:
materials = numpy.concatenate(mesh.materials)
bcond = (materials == "BOUND").astype(int)
mesh.add_cell_data("boundary_condition", bcond)


 Initial conditions can be defined as a cell data array associated to key "initial_condition" where each column of the array corresponds to a primary variable. Note that `toughio` will not write any initial condition value that is lower than the threshold flag -1.0e9.

In [5]:
centers = numpy.concatenate(mesh.centers)
incon = numpy.full((mesh.n_cells, 4), -1.0e9)
incon[:, 0] = 1.0e5 - 9810.0 * centers[:, 2]
incon[:, 1] = 0.05
incon[:, 2] = 0.0
incon[:, 3] = 10.0 - 0.025 * centers[:, 2]
mesh.add_cell_data("initial_condition", incon)


 `toughio` also recognizes the cell data keys "porosity" and "permeability" in case we want to initialize porosity and/or permeability fields (e.g. if well logs data are available). Like boundary and initial conditions, we only have to associate new cell data arrays to keys "porosity" and/or "permeability". The way these arrays are generated does not matter, they can be the results of simple interpolations (e.g. with `scipy`) or more advanced geostatistical interpolations (e.g. with `pykrige`).

 We can now write the MESH and INCON files by calling the method `to_tough`.
 Additionally, we can also pickle the final mesh for later use (reading a pickle file is much faster than reading any mesh format).

In [6]:
mesh.to_tough("MESH", incon_eos="eco2n")
mesh.write("mesh.pickle")
